### CNN-LSTM for PsP

In [ ]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dense, LSTM, concatenate, Convolution2D, MaxPooling2D, Flatten, BatchNormalization
from keras.models import Sequential, model_from_json
from keras.layers import TimeDistributed, Dropout
from keras import backend as K
from keras import optimizers

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, precision_recall_curve, average_precision_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

import scipy.io as spio
from scipy import interp
import pandas as pd
from pandas import DataFrame

# import tensorflow as tf
import numpy as np
# import dicom # install pydicom instead install dicom
import pydicom

import os.path
from matplotlib import pyplot as plt
from datetime import datetime

import itertools
import random


### For tuning for stochastic model and s
# seed=random.randint(0, 2**31 - 1)
### For reproductivity in 10-fold
# seed= 205486557
### For Benchmark and confirming for negative control with scrambling
seed=7777
### Scramble inputs for negative?
scrambleFornegative = False


#################
print(seed)
tf.random.set_seed(seed) # tf.set_random_seed(seed) # updated for TF2.2
np.random.seed(seed)
random.seed(seed)

### The number of input images
timesteps = 5

### FOV and pixel size
FOV_row = 230
FOV_col = 230
frame_row = 256
frame_col = 256
channels = 1

### The number of clinical features
nb_clinic = 0

### The number of class: PsPD versus PD
nb_class = 1

##################
def clinic_model():
    clinic = Sequential()
    clinic.add(Flatten(input_shape=[nb_clinic, 1]))
    clinic.add(Dense(4, activation='relu'))
    clinic.add(Dense(4, activation='relu'))

    return clinic

def mri_model():
    mri1 = Sequential()
    mri1.add(TimeDistributed(BatchNormalization(), input_shape=[None, frame_row, frame_col, channels]))
    mri1.add(TimeDistributed(Convolution2D(64, (2, 2), padding='same', activation='relu')))
    mri1.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    mri1.add(TimeDistributed(BatchNormalization()))
    mri1.add(TimeDistributed(Convolution2D(128, (2, 2), padding='same', activation='relu')))
    mri1.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    mri1.add(TimeDistributed(BatchNormalization()))
    mri1.add(TimeDistributed(Convolution2D(256, (2, 2), padding='same', activation='relu')))
    mri1.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    mri1.add(TimeDistributed(Flatten()))
    mri1.add(LSTM(24, activation='sigmoid'))

    mri1.add((Dense(32, activation='relu')))
    mri1.add((Dense(32, activation='relu')))

    return mri1

def mri_model (ncell):
    mri1 = Sequential()
    mri1.add(TimeDistributed(BatchNormalization(), input_shape=[None, frame_row, frame_col, channels]))
    mri1.add(TimeDistributed(Convolution2D(64, (2, 2), padding='same', activation='relu')))
    mri1.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    mri1.add(TimeDistributed(BatchNormalization()))
    mri1.add(TimeDistributed(Convolution2D(128, (2, 2), padding='same', activation='relu')))
    mri1.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
  
    mri1.add(TimeDistributed(BatchNormalization()))
    mri1.add(TimeDistributed(Convolution2D(256, (2, 2), padding='same', activation='relu')))
    mri1.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    mri1.add(TimeDistributed(Flatten()))
    mri1.add(LSTM(ncell, activation='sigmoid'))

    mri1.add(Dense(32, activation='relu'))
    mri1.add(Dense(32, activation='relu'))

    return mri1



def makemodel (ncell):
    decoder = Sequential()
    # decoder.add(Merge([mri_model(ncell), clinic_model()], mode='concat'))
    decoder.add(concatenate([mri_model(ncell), clinic_model()], mode='concat'))
    decoder.add(Dense(1, activation='sigmoid'))

    decoder.compile(loss='binary_crossentropy',
                     optimizer='sgd',
                     metrics=['accuracy'])

    return decoder

def makemodel_lr (lr):
    decoder = Sequential()
    # decoder.add(Merge([mri_model(24), clinic_model()], mode='concat'))
    decoder.add(concatenate([mri_model(24), clinic_model()], mode='concat'))
    decoder.add(Dense(1, activation='sigmoid'))
    decoder.compile(loss='binary_crossentropy',
                     optimizer=optimizers.SGD(lr=lr),
                     metrics=['accuracy'])

    return decoder


def makemodel_wo_clinic (ncell):
    decoder=mri_model(ncell)
    decoder.add(Dense(1, activation='sigmoid'))

    decoder.compile(loss='binary_crossentropy',
                     optimizer='sgd',
                     metrics=['accuracy'])


    return decoder

print('Done')

## Train

In [ ]:
# 15+2 paths
def run_train():

    TRAIN_FOLDER = '/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/train'
    x_clinic, y = load_csv(TRAIN_FOLDER)
    # x_mri1 = load_dicom(TRAIN_FOLDER, test=False) #
    # x_mri1=load_npz('Training/input_data_train.npz')
    pathFile = TRAIN_FOLDER + '/train.mat'
    x_mri1 = load_train_mat(pathFile)

    ######## Parameter Tuning ###########
    # determine_epoch(x_mri1, x_clinic, y)
    determine_batch(x_mri1, x_clinic, y)
    # determine_ncell(x_mri1, x_clinic, y)
    # determine_lr (x_mri1, x_clinic, y)

    ##########################################################################
    ## 10-fold internal validation with 1 iteration in the trainingining set  #
    ##########################################################################

    scores_roc = DataFrame()
    meanROC = list()

    scores_precision = DataFrame()
    meanPre = list()


    if scrambleFornegative==True:
        x_clinic=shuffle(x_clinic,random_state=seed)
        x_mri1=shuffle(x_mri1, random_state=seed)


    k = 3
    iter = 3


    cv = StratifiedKFold(n_splits=k, shuffle=False, random_state=seed)

    for i in range(iter):
        aucs_roc, aucs_precall= cal_auc(cv, x_clinic, x_mri1, y)
        scores_roc[str(i)] = aucs_roc
        meanROC.append(np.mean(aucs_roc))

        scores_precision[str(i)] = aucs_precall
        meanPre.append(np.mean(aucs_precall))


    if i>1:

        print(scores_roc.describe())
        scores_roc.boxplot()
        plt.show()
        plt.clf()

        print(scores_precision.describe())
        scores_precision.boxplot()
        plt.show()

        # confidence intervals
        alpha = 0.95
        p = ((1.0 - alpha) / 2.0) * 100
        lower = max(0.0, np.percentile(meanROC, p))
        p = (alpha + ((1.0 - alpha) / 2.0)) * 100
        upper = min(1.0, np.percentile(meanROC, p))
        print('AUC: Grand Mean = %.1f, %.1f confidence interval %.1f%% and %.1f%%' % (np.mean(meanROC), alpha * 100, lower * 100, upper * 100))

        p = ((1.0 - alpha) / 2.0) * 100
        lower = max(0.0, np.percentile(meanPre, p))
        p = (alpha + ((1.0 - alpha) / 2.0)) * 100
        upper = min(1.0, np.percentile(meanPre, p))
        print('AUPRC: Grand Mean = %.1f, %.1f confidence interval %.1f%% and %.1f%%' % (np.mean(meanPre), alpha * 100, lower * 100, upper * 100))

        scores_roc.to_csv('result_roc.csv')
        (scores_roc.describe()).to_csv('result_sum_roc.csv')

        scores_precision.to_csv('result_pre.csv')
        (scores_precision.describe()).to_csv('result_sum_pre.csv')



def finalize_model():

    TRAIN_FOLDER = '/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/train' 
    x_clinic, y = load_csv(TRAIN_FOLDER)
    # x_mri1 = load_dicom(TRAIN_FOLDER, test=False)
    # x_mri1=load_npz('Training/input_data_train.npz')
    pathFile = TRAIN_FOLDER + '/train.mat'
    x_mri1 = load_train_mat(pathFile)

    ## To check out negative control
    if scrambleFornegative==True:
        x_clinic=shuffle(x_clinic,random_state=seed)
        x_mri1=shuffle(x_mri1, random_state=seed)


    NCELL=24
    EPOCH=35
    BATCH=8

    decoder=makemodel(NCELL)

    architecture = decoder.to_json()
    with open('/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test/Structure.json', 'wt') as json_file:
        json_file.write(architecture)

    decoder.fit([x_mri1, x_clinic], y, batch_size=BATCH, epochs=EPOCH, shuffle=False)
    decoder.save_weights('/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test/final.hdf5')

def finalize_model_wo_clinic():
    TRAIN_FOLDER = '/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/train'
    x_clinic, y = load_csv(TRAIN_FOLDER)
    # x_mri1 = load_dicom(TRAIN_FOLDER, test=False)
    # x_mri1=load_npz('Training/input_data_train.npz')
    pathFile = TRAIN_FOLDER + '/train.mat'
    x_mri1 = load_train_mat(pathFile)
    
    NCELL=24
    EPOCH=1000
    BATCH=8

    TEST_FOLDER = '/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test'
    x_clinic, y_test = load_csv(TEST_FOLDER)
    # x_mri1 = load_dicom(TEST_FOLDER)
    # x_mri1 = load_npz('Testing/input_data_test.npz')
    pathTestFile = TEST_FOLDER + '/test.mat'
    x_test = load_test_mat(pathTestFile)
    
    decoder=makemodel_wo_clinic((NCELL))

    architecture = decoder.to_json()
    with open('/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test/model/Structure_wo.json', 'w') as json_file:
        json_file.write(architecture)
    filepath='/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test/model/best_wo.hdf5'
    callbacks = [ModelCheckpoint(filepath, verbose=0, save_best_only=True, save_weights_only=True)]
    h = decoder.fit(x_mri1, y, batch_size=BATCH, epochs=EPOCH, shuffle=False, validation_data=(x_test,y_test), verbose=1)
    decoder.save_weights('/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test/model/final_wo.hdf5')

    """
    # Trained the model
    # Compile the model
    decoder.compile(loss='binary_crossentropy',
                  optimizer=optimizers.RMSprop(lr=1e-4),
                  metrics=['acc'])
    # Train the model
    history = decoder.fit_generator(
          train_generator,
          steps_per_epoch=train_generator.samples/train_generator.batch_size ,
          epochs=20,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples/validation_generator.batch_size,
          verbose=1)
    """
    return h

def run_test():

    TEST_FOLDER = '/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test'
    x_clinic, y = load_csv(TEST_FOLDER)
    # x_mri1 = load_dicom(TEST_FOLDER)
    # x_mri1 = load_npz('Testing/input_data_test.npz')
    pathFile = TEST_FOLDER + '/test.mat'
    x_mri1 = load_test_mat(pathFile)
    
    json_file = open('/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test/model/Structure.json', 'rt')
    architecture = json_file.read()
    json_file.close()
    models = model_from_json(architecture)

    models.load_weights('/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test/model/final.hdf5')
    show_auc(models, x_mri1, x_clinic, y)

def run_test_wo_clinic():

    TEST_FOLDER = '/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test'
    x_clinic, y = load_csv(TEST_FOLDER)
    # x_mri1 = load_dicom(TEST_FOLDER)
    # x_mri1 = load_npz('Testing/input_data_test.npz')
    pathFile = TEST_FOLDER + '/test.mat'
    x_mri1 = load_test_mat(pathFile)

    json_file = open('/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test/model/Structure_wo.json', 'rt')
    architecture = json_file.read()
    json_file.close()

    models = model_from_json(architecture)

    models.load_weights('/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test/model/final_wo.hdf5')
    show_auc_wo_clinic(models, x_mri1, y)


In [ ]:
def cal_auc(cv, x_clinic, x_mri1, y):
    tprs = []
    aucs_roc = []
    aucs_precall = []
    ytests =[]
    probas = []
    precision = dict()
    recall = dict()
    average_precision = dict()


    mean_fpr = np.linspace(0, 1, 100)
    i = 0

    for train, test in cv.split(x_clinic, y):
        classifier = makemodel(24)
        classifier.fit([x_mri1[train], x_clinic[train]], y[train], batch_size=8, epochs=25, verbose=2)

        probas_ = classifier.predict([x_mri1[test], x_clinic[test]], batch_size=8)

        # Compute ROC curve
        fpr, tpr, thresholds = roc_curve(y[test], probas_)
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs_roc.append(roc_auc)
        plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC of fold %d (AUC = %0.2f)' % (i, roc_auc))

        # Compute precision-recall curve
        precision[i], recall[i], _ = precision_recall_curve(y[test], probas_)
        average_precision[i] = average_precision_score(y[test], probas_)
        ytests.append(y[test])
        probas.append(probas_)

        i += 1
        K.clear_session()    
        
        
    ################################################
    ###########   Plotting ROC curve  ##############
    ################################################
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Luck', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs_roc)
    plt.plot(mean_fpr, mean_tpr, color='b',
             label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")

    # plt.savefig('Mean ROC (AUC = %0.2f -- %0.2f).pdf' % (mean_auc, std_auc), format='pdf')
    plt.show()
    plt.clf()

    ################################################
    ##### Plotting Precision-Recall curve ##########
    ################################################

    # Compute micro-average ROC curve and ROC area
    precision["micro"], recall["micro"], _ = precision_recall_curve(np.concatenate(ytests),np.concatenate(probas))
    average_precision["micro"] = average_precision_score(y[test], probas_, average="micro")

    plt.clf()
    plt.plot(recall["micro"], precision["micro"],
             label='micro-average Precision-recall curve (AUPRC = {0:0.2f})'
                   ''.format(average_precision["micro"]))

    for j in range(i):
        plt.plot(recall[j], precision[j],
                 label='Precision-recall curve of fold {0} (AUPRC = {1:0.2f})'
                       ''.format(j, average_precision[j]))
        aucs_precall.append(average_precision[j])


    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower right")
    currenttime = datetime.now().strftime("%H%M%S")
    # plt.savefig('Precision-Recall Curve (AUPRC = %0.2f)_' % (average_precision["micro"]) + currenttime +'.pdf' , format='pdf')
    plt.show()
    plt.clf()

    return aucs_roc, aucs_precall

def show_auc(classifier, x_mri1, x_clinic, y):

    probas_ = classifier.predict([x_mri1, x_clinic], batch_size=8)

    # Compute ROC curve
    fpr, tpr, threshold = roc_curve(y, probas_)
    tpr[0]=0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, label='ROC Curve (AUC = %0.2f)' % (roc_auc), alpha=1, color='b')

    # Compute precision-recall curve
    precision, recall, _ = precision_recall_curve(y, probas_)
    average_precision = average_precision_score(y, probas_)

    ################################################
    ###########   Plotting ROC curve  ##############
    ################################################
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, alpha=0.2, color='r',label='Luck')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend(loc="lower right")

    # plt.savefig('Final ROC (AUC = %0.2f).pdf' % (roc_auc), format='pdf')
    plt.show()
    plt.clf()

    ################################################
    ##### Plotting Precision-Recall curve ##########
    ################################################

    # Compute micro-average ROC curve and ROC area

    plt.plot(recall, precision,label='Precision-Recall Curve (AUPRC = {0:0.2f})'''.format(average_precision))
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower right")
    # plt.savefig('Final Precision-Recall (AUPRC = %0.2f).pdf' % (average_precision), format='pdf')
    plt.show()
    plt.clf()


    ################################################
    ##### Confusion Matrix #########################
    ################################################


    # The optimal cut off would be where tpr is high and fpr is low
    # tpr - (1-fpr) is zero or near to zero is the optimal cut off point

    i = np.arange(len(tpr))  # index for df
    roc = pd.DataFrame(
        {'fpr': pd.Series(fpr, index=i), 'tpr': pd.Series(tpr, index=i), '1-fpr': pd.Series(1 - fpr, index=i),
         'tf': pd.Series(tpr - (1 - fpr), index=i), 'threshold': pd.Series(threshold, index=i)})
    roc.loc[(roc.tf - 0).abs().argsort()[:1]]

    #### Confusion Matrix
    roc_t = roc.loc[(roc.tf - 0).abs().argsort()[:1]]
    optimal_cutoff = list(roc_t['threshold'])
    print (optimal_cutoff)
    report = classification_report(y, probas_ >= optimal_cutoff)
    print (report)

    cm = confusion_matrix(y, probas_ >= optimal_cutoff)
    print(cm)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm)

    plt.matshow(cm, cmap=plt.cm.Blues)
    plt.colorbar()
    fmt = '.2f'
    thresh = 0.5
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.show()

def show_auc_wo_clinic (classifier, x_mri1, y):


    probas_ = classifier.predict([x_mri1], batch_size=8)
    # Compute ROC curve
    fpr, tpr, threshold = roc_curve(y, probas_)
    tpr[0]=0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, label='ROC Curve (AUC = %0.2f)' % (roc_auc), alpha=1, color='orange')

    # Compute precision-recall curve
    precision, recall, _ = precision_recall_curve(y, probas_)
    average_precision = average_precision_score(y, probas_)

    ################################################
    ###########   Plotting ROC curve  ##############
    ################################################
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, alpha=0.2, color='r',label='Random')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    #plt.title('Receiver Operating Characteristic Curve')
    plt.legend(loc="lower right")
    plt.gca().set_aspect(0.75, adjustable='box')
    
    # plt.savefig('Final ROC wo clinic (AUC = %0.2f).pdf' % (roc_auc), format='pdf')
    plt.show()
    plt.clf()

    ################################################
    ##### Plotting Precision-Recall curve ##########
    ################################################

    # Compute micro-average ROC curve and ROC area

    plt.plot(recall, precision,label='Precision-Recall Curve (AUC = {0:0.2f})'''.format(average_precision), color='orange')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower right")
    # plt.savefig('Final Precision-Recall wo clinic(AUC = %0.2f).pdf' % (average_precision), format='pdf')
    plt.show()
    plt.clf()


    ################################################
    ##### Confusion Matrix #########################
    ################################################


    # The optimal cut off would be where tpr is high and fpr is low
    # tpr - (1-fpr) is zero or near to zero is the optimal cut off point

    i = np.arange(len(tpr))  # index for df
    roc = pd.DataFrame(
        {'fpr': pd.Series(fpr, index=i), 'tpr': pd.Series(tpr, index=i), '1-fpr': pd.Series(1 - fpr, index=i),
         'tf': pd.Series(tpr - (1 - fpr), index=i), 'threshold': pd.Series(threshold, index=i)})
    roc.loc[(roc.tf - 0).abs().argsort()[:1]]

    #### Confusion Matrix
    roc_t = roc.loc[(roc.tf - 0).abs().argsort()[:1]]
    optimal_cutoff = list(roc_t['threshold'])
    print (optimal_cutoff)
    report = classification_report(y, probas_ >= optimal_cutoff)
    print (report)

    cm = confusion_matrix(y, probas_ >= optimal_cutoff)
    print(cm)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm)

    plt.matshow(cm, cmap=plt.cm.Oranges)
    plt.colorbar()
    fmt = '.2f'
    thresh = 0.5
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.show()

def plot_history (history):

    plt.figure(1)
    plt.plot(history.history['loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.plot(history.history['val_loss'])
    plt.title('model train vs validation loss')
    plt.legend(['train', 'validation'], loc='upper right')
    plt.show()


def load_csv(path):
    pathfile = path+'/y.csv'
    dataframe = pd.read_csv(pathfile)
    dataset = dataframe.values


    X = np.array (dataset[:, 0:nb_clinic+1].astype(float))
    Y = np.array(dataset[:, nb_clinic+1])

    scaler = MinMaxScaler (feature_range=(0,1))
    scalerX = scaler.fit(X)
    normalizedX = scalerX.transform(X)

    X = normalizedX.reshape(normalizedX.shape[0], normalizedX.shape[1], 1)
    return X,Y

def load_dicom(path, test=False):


    mark = ['0000', '0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008']
    n = len(os.listdir(path)) - 1
    j = 0

    stack = np.zeros((timesteps, frame_row, frame_col))
    data = np.zeros((n, timesteps, frame_row, frame_col))

    with tf.Session() as sess:
        for ptnum in os.listdir(path):
            if ptnum == 'sample.csv':
                continue
            else:
                for i in mark:
                    # dcm_struct=dicom.read_file(path + '/' + ptnum + '/' + ptnum + '_' + i + '.dcm')
                    dcm_struct=pydicom.read_file(path + '/' + ptnum + '/' + ptnum + '_' + i + '.dcm')
                    pixel_size=dcm_struct.PixelSpacing[0]
                    img = dcm_struct.pixel_array
                    plt.imshow(img, cmap=plt.cm.gray)
                    img = img.reshape(img.shape[0],img.shape[1],1)
                    img = img.astype(int)
                    print(i,j)
                    img = tf.image.resize_image_with_crop_or_pad(img, int(round(FOV_row/pixel_size)), int(round(FOV_col/pixel_size)))
                    img = tf.image.resize_images(img, size=[frame_row, frame_col], method=tf.image.ResizeMethod.BICUBIC)
                    img = tf.image.per_image_standardization(img)

                    img_array = sess.run(img)
                    img_array = img_array.reshape(img.shape[0], img.shape[1])
                    plt.imshow(img_array, cmap=plt.cm.gray)
                    stack[int(i), :, :] = img_array

            data [j,:,:,:] = stack
            j = j+1


    data = data.reshape(data.shape[0], data.shape[1], data.shape[2], data.shape[3], channels)

    if test==True:
        np.savez_compressed('input_data_test.npz', data)
    else:
        np.savez_compressed('input_data_train.npz', data)

    # show_slice(data[0][4])

    return data

def load_npz(file):
    data=np.load(file)
    data=data['arr_0']
    return data

def load_test_mat(file):
    fileDir = '/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/test'
    fileName = fileDir + '/test.mat'
    data = spio.loadmat(fileName)['IMG']
    data = np.ascontiguousarray(data.T)
    data = np.swapaxes(data,2,4)
    # data = data[:,:,:,:,np.newaxis]
    return data

def load_train_mat(file):
    fileDir = '/data/jslee/PsP/PsPData/MatData_lstm/PsP43/Modality_7/CV1/train'
    fileName = fileDir + '/train.mat'
    data = spio.loadmat(fileName)['IMG']
    data = np.ascontiguousarray(data.T)
    data = np.swapaxes(data,2,4)
    # data = data[:,:,:,:,np.newaxis]
    return data

def show_slice(arr, value_range=None):
    if len(list(arr.shape)) > 2:
        arr2 = arr.copy()
        arr2 = np.reshape(arr, (arr.shape[0], arr.shape[1]))
    else:
        arr2 = arr

    dpi = 80
    margin = 0.05  # (5% of the width/height of the figure...)
    xpixels, ypixels = arr2.shape[0], arr2.shape[1]

    figsize = (1 + margin) * ypixels / dpi, (1 + margin) * xpixels / dpi

    fig = plt.figure(figsize=figsize, dpi=dpi)
    # Make the axis the right size...
    ax = fig.add_axes([margin, margin, 1 - 2 * margin, 1 - 2 * margin])


    if value_range is None:
        plt.imshow(arr2, cmap=plt.cm.gray)
    else:
         plt.imshow(arr2, vmin=value_range[0], vmax=1, cmap=plt.cm.gray, interpolation='none')

    plt.show()

def random_forest():

    TRAIN_FOLDER = 'C:/Users/JBS/Desktop/PsPDtrain_1mm(SNUH)'
    TEST_FOLDER = 'C:/Users/JBS/Desktop/PsPDtest_1mm(SNUBH)'
    testX_clinic, testY = load_csv(TEST_FOLDER)
    trainX_clinic, trainY = load_csv(TRAIN_FOLDER)

    x1=trainX_clinic.shape[0]
    x2=trainX_clinic.shape[1]
    trainX_clinic= trainX_clinic.reshape(x1,x2)

    x1=testX_clinic.shape[0]
    x2=testX_clinic.shape[1]
    testX_clinic= testX_clinic.reshape(x1,x2)



#### Internal Val in the training set
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
    rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=seed)
    results = cross_val_score(rf, trainX_clinic, trainY, cv=cv)
    print(results.mean())

    tprs = []
    aucs_roc = []
    aucs_precall = []
    ytests =[]
    probas = []
    precision = dict()
    recall = dict()
    average_precision = dict()


    mean_fpr = np.linspace(0, 1, 100)
    i = 0

    for train, test in cv.split(trainX_clinic, trainY):
        rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=seed)
        rf.fit(trainX_clinic[train], trainY[train])
        probas_ = rf.predict(trainX_clinic[test])

        # Compute ROC curve
        fpr, tpr, thresholds = roc_curve(trainY[test], probas_)
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs_roc.append(roc_auc)
        plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC of fold %d (AUC = %0.2f)' % (i, roc_auc))

        # Compute precision-recall curve
        precision[i], recall[i], _ = precision_recall_curve(trainY[test], probas_)
        average_precision[i] = average_precision_score(trainY[test], probas_)
        ytests.append(trainY[test])
        probas.append(probas_)

        i += 1



    ################################################
    ###########   Plotting ROC curve  ##############
    ################################################
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Luck', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs_roc)
    plt.plot(mean_fpr, mean_tpr, color='b',
             label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")

    # plt.savefig('RF_internal_Mean ROC (AUC = %0.2f -- %0.2f).pdf' % (mean_auc, std_auc), format='pdf')
    plt.show()
    plt.clf()



#### External Val in the testing set
    rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=seed)
    rf.fit(trainX_clinic, trainY)

    probas_ = rf.predict(testX_clinic)
    accuracy = accuracy_score(testY, probas_)

    print(f'Out-of-bag score estimate: {rf.oob_score_:.3}')
    print(f'Mean accuracy score: {accuracy:.3}')


   # Compute ROC curve
    fpr, tpr, threshold = roc_curve(testY, probas_)
    tpr[0]=0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, label='ROC Curve (AUC = %0.2f)' % (roc_auc), alpha=1, color='g')

    # Compute precision-recall curve
    precision, recall, _ = precision_recall_curve(testY, probas_)
    average_precision = average_precision_score(testY, probas_)

    ################################################
    ###########   Plotting ROC curve  ##############
    ################################################
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, alpha=0.2, color='r',label='Luck')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend(loc="lower right")

    # plt.savefig('RF_ROC (AUC = %0.2f).pdf' % (roc_auc), format='pdf')
    plt.show()
    plt.clf()

    ################################################
    ##### Plotting Precision-Recall curve ##########
    ################################################

    # Compute micro-average ROC curve and ROC area

    plt.plot(recall, precision,label='Precision-Recall Curve (AUPRC = {0:0.2f})'''.format(average_precision), color='green')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower right")
    # plt.savefig('RF_Precision-Recall (AUPRC = %0.2f).pdf' % (average_precision), format='pdf')
    plt.show()
    plt.clf()


    ################################################
    ##### Confusion Matrix #########################
    ################################################


    # The optimal cut off would be where tpr is high and fpr is low
    # tpr - (1-fpr) is zero or near to zero is the optimal cut off point

    i = np.arange(len(tpr))  # index for df
    roc = pd.DataFrame(
        {'fpr': pd.Series(fpr, index=i), 'tpr': pd.Series(tpr, index=i), '1-fpr': pd.Series(1 - fpr, index=i),
         'tf': pd.Series(tpr - (1 - fpr), index=i), 'threshold': pd.Series(threshold, index=i)})
    roc.loc[(roc.tf - 0).abs().argsort()[:1]]

    #### Confusion Matrix
    roc_t = roc.loc[(roc.tf - 0).abs().argsort()[:1]]
    optimal_cutoff = list(roc_t['threshold'])
    print (optimal_cutoff)
    report = classification_report(testY, probas_ >= optimal_cutoff)
    print (report)

    cm = confusion_matrix(testY, probas_ >= optimal_cutoff)
    print(cm)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm)

    plt.matshow(cm, cmap=plt.cm.Greens)
    plt.colorbar()
    fmt = '.2f'
    thresh = 0.5
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    # plt.savefig('Confusion matrix.pdf', format='pdf')
    plt.show()

print('Done')

In [ ]:
if __name__ == '__main__':

###### Model 1 ######
     # run_train()
     # finalize_model()
     # run_test()

###### Model 2 ######
     h = finalize_model_wo_clinic()
     #run_test_wo_clinic()

###### Model 3 ######
     # random_forest()

In [ ]:
# Check Performance
history = h
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

### Run Test

In [ ]:
run_test_wo_clinic()

### 1. ROC curve & Load Model ===================

In [ ]:
# move test/train folders to C:\Users\leejoons\PROJECTS_DATA\1-1_PsP\PsPData\ImageData
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, precision_recall_curve
from sklearn.metrics import f1_score, average_precision_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model, Sequential, model_from_json
from itertools import cycle
import math
import scipy.io as spio
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

fpr = dict()
tpr = dict()
threshold = dict()
roc_auc = dict()
accuracy = dict()
interval = dict()
optimal_cutoff = dict()
y_true = dict()
y_pred = dict()
probability = dict()

n_classes = 3
nb_clinic = 0
print('done')

### Load Model ---- change n_cv

In [ ]:
# move test folder to 'C:/Users/leejoons/PROJECTS_DATA/1-1_PsP/PsPData/MatData/test'
n_cv = 0 # 0: cv1 k-fold cv

nb_clinic = 0
def load_csv(path):
    pathfile = path+'/y.csv'
    dataframe = pd.read_csv(pathfile)
    dataset = dataframe.values
    X = np.array (dataset[:, 0:nb_clinic+1].astype(float))
    Y = np.array(dataset[:, nb_clinic+1])
    scaler = MinMaxScaler (feature_range=(0,1))
    scalerX = scaler.fit(X)
    normalizedX = scalerX.transform(X)
    X = normalizedX.reshape(normalizedX.shape[0], normalizedX.shape[1], 1)
    return X,Y

def load_test_mat(file):
    fileDir = 'E:/PROJECTS/1_RAD/PsP/PsPData/MatData_lstm/PsP43/Modality_3/CV1/test'
    fileName = fileDir + '/test.mat'
    data = spio.loadmat(fileName)['IMG']
    data = np.ascontiguousarray(data.T)
    data = np.swapaxes(data,2,4)
    # data = data[:,:,:,:,np.newaxis]
    return data

def run_test_wo_clinic():
    TEST_FOLDER = 'E:/PROJECTS/1_RAD/PsP/PsPData/MatData_lstm/PsP43/Modality_3/CV1/test'
    x_clinic, y = load_csv(TEST_FOLDER)
    pathFile = TEST_FOLDER + '/test.mat'
    x_mri1 = load_test_mat(pathFile)
    json_file = open('E:/PROJECTS/1_RAD/PsP/PsPData/MatData_lstm/PsP43/Modality_3/CV1/test/model/Structure_wo.json', 'rt')
    architecture = json_file.read()
    json_file.close()
    models = model_from_json(architecture)
    #models.load_weights('C:/Users/leejoons/PROJECTS_DATA/1-1_PsP/PsPData/MatData/test/best_wo.hdf5')
    models.load_weights('E:/PROJECTS/1_RAD/PsP/PsPData/MatData_lstm/PsP43/Modality_3/CV1/test/model/final_wo.hdf5')

    return models, x_mri1, y

models, x_mri1, y = run_test_wo_clinic()
print(len(y))

#### Assign Variables

In [ ]:
# Compute ROC curve
i = n_cv # ith in k-folds 0:cv1, 1:cv2, 2:cv3 <-------------------- modify fold number 

prob = models.predict([x_mri1], batch_size=8)
fpr[i], tpr[i], threshold[i] = roc_curve(y, prob)
roc_auc[i] = auc(fpr[i], tpr[i])
y_true[i] = y
probability[i] = prob

print('# of var = ', len(roc_auc))

#### Results

In [ ]:
# The optimal cut off would be where tpr is high and fpr is low
# tpr - (1-fpr) is zero or near to zero is the optimal cut off point
# import pandas as pd

k = np.arange(len(tpr[i]))  # index for df
roc = pd.DataFrame(
    {'fpr': pd.Series(fpr[i], index=k), 'tpr': pd.Series(tpr[i], index=k), '1-fpr': pd.Series(1 - fpr[i], index=k),
         'tf': pd.Series(tpr[i] - (1 - fpr[i]), index=k), 'threshold': pd.Series(threshold[i], index=k)})

roc.loc[(roc.tf - 0).abs().argsort()[:1]]

#### Confusion Matrix
roc_t = roc.loc[(roc.tf - 0).abs().argsort()[:1]]
optimal_cutoff[i] = list(roc_t['threshold'])
print (optimal_cutoff[i])
# print(probas_[0:5])
report = classification_report(y_true[i], probability[i] >= optimal_cutoff[i])
print(report)

cm = confusion_matrix(y_true[i], probability[i] >= optimal_cutoff[i])
print(cm)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm)

#### Something wrong

In [ ]:
probs = np.squeeze(prob)
probs[probs >= optimal_cutoff[i]] = 1
probs[probs < optimal_cutoff[i]] = 0
probs = probs.astype(np.int64)
y_pred[i] = probs

#### 95% C.I. for accuracy

In [ ]:
# import math
# from sklearn.metrics import f1_score
accuracy[i] = f1_score(y, prob >= optimal_cutoff[i], average='micro')
interval[i] = 1.96 * math.sqrt( (accuracy[i] * (1 - accuracy[i])) / len(y))
print('95% C.I. = {:.4f} \u00B1 {:.4f}'.format(accuracy[i], interval[i]))

a = accuracy[i] - interval[i]
b = accuracy[i] + interval[i]
print('95% C.I. = [{:.4f} - {:.4f}]'.format(a,b))

#### Save y variables to Excel

In [ ]:
i = n_cv
import numpy as np
np.savetxt("y_prob.csv", probability[i], delimiter=",")
np.savetxt("y_true.csv", y_true[i], delimiter=",")
print(roc_auc[i])

In [ ]:
# for inverse value
A = tpr[i]
B = fpr[i]
tpr[i] = B
fpr[i] = A
roc_auc[i] = auc(fpr[i], tpr[i])
print(roc_auc[i])

#### -------> go to R for 95% IC for AUC

#### 95% C.I using bootstrap

In [ ]:
import numpy as np
from scipy.stats import sem
from sklearn.metrics import roc_auc_score

# y_pred = np.array([0.21, 0.32, 0.63, 0.35, 0.92, 0.79, 0.82, 0.99, 0.04])
# y_true = np.array([0,    1,    0,    0,    1,    1,    0,    1,    0   ])
y_pr = y_pred[i]
y_tr = y_true[i]

print("Original ROC area: {:0.3f}".format(roc_auc_score(y_tr, y_pr)))

n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.random_integers(0, len(y_pr) - 1, len(y_pr))
    if len(np.unique(y_tr[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(y_tr[indices], y_pr[indices])
    bootstrapped_scores.append(score)
    # print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

plt.hist(bootstrapped_scores, bins=50)
plt.title('Histogram of the bootstrapped ROC AUC scores')
plt.show()

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))



#### -----> Load Next Model for the Multi-plot

#### Multiple ROC Curves

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle

n_classes = 3

# Plot all ROC curves
plt.figure()
colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], lw=1, label='ROC Curve (AUC = %0.2f)' % (roc_auc[i]), alpha=1, color=color)
    print('AUC_{} = {:.4f}' .format(i, roc_auc[i]))
print('Mean AUC = {:.4f}'.format((roc_auc[0]+roc_auc[1]+roc_auc[2])/3))

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, alpha=0.2, color='r',label='Luck')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc="lower right")

# plt.savefig('Final ROC wo clinic (AUC = %0.2f).pdf' % (roc_auc), format='pdf')
plt.show()
plt.clf()

#### Box plot

In [ ]:
dataset = np.array([roc_auc[0], roc_auc[1], roc_auc[2]])
print(dataset)
dataset.shape

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# plt.rcParams["figure.figsize"] = (6,4)
# dataset = np.array([[1,2,3,3,4,4,2], [3,2,5,3,3,3,3], [3,4,3,5,2,2,3]])
dataset = np.array([roc_auc[0], roc_auc[1], roc_auc[2]])
# df = pd.DataFrame(dataset, columns=['T1', 'T1post', 'T2', 'FLAIR', 'ADC', 'T1post-T1', 'T2-FLAIR'])
df = pd.DataFrame(dataset, columns=['CNN-LSTM 5'])
ax = df.plot.box(grid='True', title='Boxplot for T1 post', colormap='jet')
# ax = df.plot(lw=1,colormap='jet',marker='.',markersize=10,title='Boxplot for T1 Post')
# set labels for both axes
ax.set(ylabel='AUC')

#### Save Variables

In [ ]:
import pickle

# Saving the objects:
with open('Var_7Mod_LSTM_9var.pkl', 'wb') as f:  
    pickle.dump([fpr, tpr, threshold, roc_auc, accuracy, interval, optimal_cutoff, y_true, probability], f)

#### Load Variables

In [ ]:
import pickle
# Getting back the objects:
with open('Var_5Mod_LSTM_9var.pkl', 'rb') as f:  
    fpr, tpr, threshold, roc_auc, accuracy, interval, optimal_cutoff, y_true, probability = pickle.load(f)

In [ ]:
%whos dict

In [ ]:
print(probability)

In [ ]:
print(y_true[1])
print(y_pred[1])
np.savetxt("foo.csv", y_pred[2], delimiter=",")

In [ ]:
import numpy as np
np.shape(y_pred)